# Logger Wrapper personalization/configuration

We will see on this notebook how to personalize the logger wrapper defined by sinergym.

In [5]:
import gym
import numpy as np
import sinergym
from sinergym.utils.wrappers import (LoggerWrapper, MultiObsWrapper,
                                     NormalizeObservation)
from sinergym.utils.constants import RANGES_5ZONE

## Step 1 Inherit and modify the CSVloger

First we need to change the CSV logger to modify the values writen into the file on the funtion create_row_contents

In [6]:

from sinergym.utils.logger import CSVLogger
from typing import Any, Dict, Optional, Sequence, Tuple, Union, List

class CustomCSVLogger(CSVLogger):

    def __init__(
            self,
            monitor_header: str,
            progress_header: str,
            log_progress_file: str,
            log_file: Optional[str] = None,
            flag: bool = True):
        super(CustomCSVLogger, self).__init__(monitor_header,progress_header,log_progress_file,log_file,flag)
        self.last_10_steps_reward = [0]*10

    def _create_row_content(
            self,
            obs: List[Any],
            action: Union[int, np.ndarray, List[Any]],
            reward: Optional[float],
            done: bool,
            info: Optional[Dict[str, Any]]) -> List:
            
        if reward is not None:
            self.last_10_steps_reward.pop(0)
            self.last_10_steps_reward.append(reward)

        if info is None:  # In a reset
            return [0] + list(obs) + list(action) + \
                [0, reward, np.mean(self.last_10_steps_reward), None, None, None, done]
        else:
            return [
                info['timestep']] + list(obs) + list(action) + [
                info['time_elapsed'],
                reward,
                np.mean(self.last_10_steps_reward),
                info['total_power_no_units'],
                info['comfort_penalty'],
                info['abs_comfort'],
                done]

    

## Step 2 Intanciate the LoggerWrapper


now we need to instantiate the loggerwrapper and specify the new headers of our file and the csvlogger class we want to use.

In [7]:
env=gym.make('Eplus-demo-v1')
env=LoggerWrapper(env,logger_class=CustomCSVLogger,monitor_header = ['timestep'] + env.variables['observation'] +
                env.variables['action'] + ['time (seconds)', 'reward', '10-mean-reward',
                'power_penalty', 'comfort_penalty', 'done'])

[2022-06-21 09:33:58,455] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf ExternalInterface object if it is not present...
[2022-06-21 09:33:58,456] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2022-06-21 09:33:58,458] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf OutPut:Variable and variables XML tree model for BVCTB connection.
[2022-06-21 09:33:58,460] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Setting up extra configuration in building model if exists...


Now, you can see in Sinergym output folder that you will have available `progress.csv` file and `monitor.csv` files in each episode.

In [8]:
for i in range(1):
    obs = env.reset()
    rewards = []
    done = False
    current_month = 0
    while not done:
        a = env.action_space.sample()
        obs, reward, done, info = env.step(a)
        rewards.append(reward)
        if info['month'] != current_month:  # display results every month
            current_month = info['month']
            print('Reward: ', sum(rewards), info)
    print('Episode ', i, 'Mean reward: ', np.mean(
        rewards), 'Cumulative reward: ', sum(rewards))
env.close()

[2022-06-21 09:34:00,784] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2022-06-21 09:34:00,794] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/Eplus-env-demo-v1-res2/Eplus-env-sub_run1


Reward:  -0.3808358083250144 {'timestep': 1, 'time_elapsed': 900, 'year': 1991, 'month': 1, 'day': 1, 'hour': 0, 'total_power': 7616.716166500288, 'total_power_no_units': -0.7616716166500288, 'comfort_penalty': -0.0, 'abs_comfort': 0.0, 'temperatures': [20.99998783039325], 'out_temperature': 1.8, 'action_': [21, 21]}
Reward:  -1817.7037351644783 {'timestep': 2976, 'time_elapsed': 2678400, 'year': 1991, 'month': 2, 'day': 1, 'hour': 0, 'total_power': 8278.265126379221, 'total_power_no_units': -0.8278265126379222, 'comfort_penalty': -0.0, 'abs_comfort': 0.0, 'temperatures': [20.09635860317218], 'out_temperature': -7.0, 'action_': [20, 25]}
Reward:  -3569.893869126496 {'timestep': 5664, 'time_elapsed': 5097600, 'year': 1991, 'month': 3, 'day': 1, 'hour': 0, 'total_power': 833.2920927358493, 'total_power_no_units': -0.08332920927358493, 'comfort_penalty': -0.43408478076705137, 'abs_comfort': 0.43408478076705137, 'temperatures': [19.56591521923295], 'out_temperature': 8.1, 'action_': [16, 2

[2022-06-21 09:34:15,548] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 
